<a href="https://colab.research.google.com/github/SamuelLachance/NHL3.0/blob/main/NHL_PRED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from optparse import OptionContainer
from textblob.en.np_extractors import filter_insignificant
from http.cookiejar import LoadError
import os
import requests
from textblob import TextBlob
import datetime
import re
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import statsmodels.api as sm
import pickle
import time
import random
import json
import unicodedata
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.ensemble import GradientBoostingClassifier
import warnings
import xgboost as xgb
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import GammaRegressor
from sklearn.linear_model import PoissonRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

warnings.filterwarnings("ignore")

#remove accents

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii

def weird_division(x,y):
    if y == 0:
        return 0
    return x / y

#get the time
yesterday = (datetime.date.today()- datetime.timedelta(days=1)).strftime('%Y-%m-%d')
today = datetime.date.today().strftime('%Y-%m-%d')
tomorrow = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
year = 20222023
# Retrieve the list of game IDs for the season


urlHist = 'https://www.naturalstattrick.com/games.php?fromseason=20182019&thruseason=20212022&stype=2&sit=all&loc=B&team=All&rate=n'
req = requests.get(urlHist)

soup = BeautifulSoup(req.content)

top = list(soup.children)

body = soup.find('body')

body.find_all('th')

columns = [item.text for item in body.find_all('th')]
columns

body.find_all('td')

data = [e.text for e in body.find_all('td')]

start = 0
table= []

while start+len(columns) <= len(data):
    player = []
    #use length of columns as iteration stop point to get list of info for 1 player 
    for i in range(start,start+len(columns)):
        player.append(data[i])
    #add player row to list
    table.append(player)
    #start at next player
    start += len(columns)

df = pd.DataFrame(table, columns = columns, dtype = 'float').set_index('')

df.head()

urlHist2 = 'https://www.naturalstattrick.com/teamtable.php?fromseason=20222023&thruseason=20222023&stype=2&sit=all&score=all&rate=y&team=all&loc=B&gpf=c&gp=10&fd=&td='
req2 = requests.get(urlHist2)

soup2 = BeautifulSoup(req2.content)

top2 = list(soup2.children)

body2 = soup2.find('body')

body2.find_all('th')

columns2 = [item.text for item in body2.find_all('th')]
columns2

body2.find_all('td')

data2 = [e.text for e in body2.find_all('td')]

start2 = 0
table2 = []

while start2+len(columns2) <= len(data2):
    player = []
    #use length of columns as iteration stop point to get list of info for 1 player 
    for i in range(start2,start2+len(columns2)):
        player.append(data2[i])
    #add player row to list
    table2.append(player)
    #start at next player
    start2 += len(columns2)

df2 = pd.DataFrame(table2, columns = columns2, dtype = 'float').set_index('')

df2.head()

urlHist3 = 'https://www.naturalstattrick.com/teamtable.php?fromseason=20222023&thruseason=20222023&stype=2&sit=all&score=all&rate=y&team=all&loc=B&gpf=410&fd=&td='
req3 = requests.get(urlHist3)

soup3 = BeautifulSoup(req3.content)

top3 = list(soup3.children)

body3 = soup3.find('body')

body3.find_all('th')

columns3 = [item.text for item in body3.find_all('th')]
columns3

body3.find_all('td')

data3 = [e.text for e in body3.find_all('td')]

start3 = 0
table3 = []

while start3+len(columns3) <= len(data3):
    player = []
    #use length of columns as iteration stop point to get list of info for 1 player 
    for i in range(start3,start3+len(columns3)):
        player.append(data3[i])
    #add player row to list
    table3.append(player)
    #start at next player
    start3 += len(columns3)

df3 = pd.DataFrame(table3, columns = columns3, dtype = 'float').set_index('')

df3.head()


# Diviser les données en données d'entraînement et de test
X = df[['xGF', 'xGA','HDCF', 'HDCA','MDCF','MDCA','HDSF','HDSA','MDSF','MDSA','SH%','SV%']]  # Les variables prédictives
y_gf = df['GF']  # La variable cible "GF"
y_ga = df['GA']  # La variable cible "GA"

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_gf_train, y_gf_test, y_ga_train, y_ga_test = train_test_split(X, y_gf, y_ga, test_size=0.2, random_state=42)

param_grid_rf = {
    'n_estimators': 300,
    'max_depth': None,
    'min_samples_split': 5
}

param_grid_xgb = {
    'max_depth': 3,
    'learning_rate': 0.1,
    'n_estimators': 1000,
    'gamma': 0
}

param_grid_gbm = {
    'learning_rate': 0.1,
    'n_estimators': 1000,
    'max_depth': 3,
    'min_samples_split': 5,
    'min_samples_leaf': 2
}
# Train the random forest model on the training data for "GF"
rf_gf = RandomForestRegressor(**param_grid_rf)
rf_gf.fit(X_train, y_gf_train)

# Train the random forest model on the training data for "GA"
rf_ga = RandomForestRegressor(**param_grid_rf)
rf_ga.fit(X_train, y_ga_train)

# Train the XGBoost model on the training data for "GF"
xgb_gf = XGBRegressor(**param_grid_xgb)
xgb_gf.fit(X_train, y_gf_train)

# Train the XGBoost model on the training data for "GA"
xgb_ga = XGBRegressor(**param_grid_xgb)
xgb_ga.fit(X_train, y_ga_train)

# Train the gradient boosting model on the training data for "GF"
gbm_gf = GradientBoostingRegressor(**param_grid_gbm)
gbm_gf.fit(X_train, y_gf_train)

# Train the gradient boosting model on the training data for "GA"
gbm_ga = GradientBoostingRegressor(**param_grid_gbm)
gbm_ga.fit(X_train, y_ga_train)

# Create an instance of SVR model for "GF"
svr_gf = SVR(kernel='rbf', C=1e3, gamma='scale')
svr_gf.fit(X_train, y_gf_train)


# Create an instance of SVR model for "GA"
svr_ga = SVR(kernel='rbf', C=1e3, gamma='scale')
svr_ga.fit(X_train, y_ga_train)

def find_best_W(X, y, models, weights=None, n_iterations=1000):
    
    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train each model on the training set
    preds_train = []
    for model in models:
        model.fit(X_train, y_train)
        preds_train.append(model.predict(X_train))
    
    # If no weights are provided, initialize them to equal values
    if weights is None:
        weights = np.full(len(models), 1/len(models))
    
    # Define the objective function as the validation set performance
    def objective_function(weights):
        preds_val = [model.predict(X_val) for model in models]
        combined_preds = np.average(preds_val, axis=0, weights=weights)
        return mean_squared_error(y_val, combined_preds)
    
    # Run the optimization algorithm
    best_weights = weights
    best_score = objective_function(weights)
    for i in range(n_iterations):
        new_weights = [w + random.uniform(-0.1, 0.1) for w in weights]  # perturb the weights
        new_weights = np.clip(new_weights, 0, 1)  # ensure weights are between 0 and 1
        score = objective_function(new_weights)
        if score < best_score:
            best_weights = new_weights
            best_score = score
        weights = new_weights
    
    # Normalize the weights
    weights_sum = sum(best_weights)
    best_weights = [w/weights_sum for w in best_weights]
    
    # Create a dictionary of model weights
    model_weights = {}
    for i, model in enumerate(models):
        model_weights[type(model).__name__] = best_weights[i]
    
    return model_weights

#best_weights = find_best_W(X, y_gf, [xgb_gf, gbm_gf, svr_gf], None, 1000)

#print(best_weights)

# Combine the models
def Test_model_GF(X):
    
    # Use the trained models to make predictions for "GF"
    rf_gf_pred = rf_gf.predict(X)
    xgb_gf_pred = xgb_gf.predict(X)
    gbm_gf_pred = gbm_gf.predict(X)
    svr_gf_pred = svr_gf.predict(X)
    
    # Combine the predictions using a weighted average
    gf_pred = (0.09264992620841171*xgb_gf_pred) + (0.6658086047607201*gbm_gf_pred)  + (0.24154146903086823*svr_gf_pred) #(rf_gf_pred + xgb_gf_pred + svr_gf_pred)/3 #(0.22221016502179602*rf_gf_pred) + (0.250744771234347*xgb_gf_pred) + (0.20498370405959596*gbm_gf_pred) + (0.3220613596842609*svr_gf_pred)
    
    return gf_pred

def Test_model_GA(X):
    # Use the trained models to make predictions
    # Use the trained models to make predictions for "GA"
    rf_ga_pred = rf_ga.predict(X)
    xgb_ga_pred = xgb_ga.predict(X)
    gbm_ga_pred = gbm_ga.predict(X)
    svr_ga_pred = svr_ga.predict(X)
    
    # Combine the predictions using a weighted average
    ga_pred = (0.6097886212917437*xgb_ga_pred) + (0.3902113787082563*svr_ga_pred) #(rf_ga_pred + xgb_ga_pred + svr_ga_pred)/3 #(0.27043590130033*rf_ga_pred) + (0.2693404847288102*xgb_ga_pred) + (0.18025115844990966*gbm_ga_pred) + (0.27997245552095007*svr_ga_pred)
    
    return ga_pred

def combined_model_goals(home_xGF,home_xGA,home_HDCF, home_HDCA, home_MDCF, home_MDCA,home_HDSF,home_HDSA,home_MDSF,home_MDSA,home_SH,home_SV):
    # Convert the input arguments into a numpy array
    X = np.array([[home_xGF,home_xGA,home_HDCF, home_HDCA, home_MDCF, home_MDCA,home_HDSF,home_HDSA,home_MDSF,home_MDSA,home_SH,home_SV]])
    # Use the trained models to make predictions for "GF"
    rf_gf_pred = rf_gf.predict(X)
    xgb_gf_pred = xgb_gf.predict(X)
    gbm_gf_pred = gbm_gf.predict(X)
    svr_gf_pred = svr_gf.predict(X)
    
    # Use the trained models to make predictions for "GA"
    rf_ga_pred = rf_ga.predict(X)
    xgb_ga_pred = xgb_ga.predict(X)
    gbm_ga_pred = gbm_ga.predict(X)
    svr_ga_pred = svr_ga.predict(X)
    
    # Combine the predictions using a weighted average
    gf_pred = (0.09264992620841171*xgb_gf_pred) + (0.6658086047607201*gbm_gf_pred)  + (0.24154146903086823*svr_gf_pred) #(0.22221016502179602*rf_gf_pred) + (0.250744771234347*xgb_gf_pred) + (0.20498370405959596*gbm_gf_pred) + (0.3220613596842609*svr_gf_pred)
    ga_pred = (0.6097886212917437*xgb_ga_pred) + (0.3902113787082563*svr_ga_pred) #(0.27043590130033*rf_ga_pred) + (0.2693404847288102*xgb_ga_pred) + (0.18025115844990966*gbm_ga_pred) + (0.27997245552095007*svr_ga_pred)
    
    return gf_pred, ga_pred

y_pred_combined_GF = Test_model_GF(X_test)
y_pred_combined_GA = Test_model_GA(X_test)
accuracy_GF = accuracy_score(y_gf_test, np.round(y_pred_combined_GF))
accuracy_GA = accuracy_score(y_ga_test, np.round(y_pred_combined_GA))
print("Accuracy_GF: ", accuracy_GF)
print("Accuracy_GA: ", accuracy_GA)

mse_GF = mean_squared_error(y_gf_test, y_pred_combined_GF)
rmse_GF = np.sqrt(mse_GF)
r2_GF = r2_score(y_gf_test, y_pred_combined_GF)

mse_GA = mean_squared_error(y_ga_test, y_pred_combined_GA)
rmse_GA = np.sqrt(mse_GA)
r2_GA = r2_score(y_ga_test, y_pred_combined_GA)

print("MSE_GF: ", mse_GF)
print("RMSE_GF: ", rmse_GF)
print("R-squared_GF: ", r2_GF)

print("MSE_GA: ", mse_GA)
print("RMSE_GA: ", rmse_GA)
print("R-squared_GA: ", r2_GA)


# Récupérer les matchs à venir à partir de l'API NHL
response = requests.get("https://statsapi.web.nhl.com/api/v1/schedule?date=" + today + "&expand=schedule.linescore")
schedule = response.json()['dates'][0]['games']

# Prédire les résultats de chaque match
for game in schedule:
    home_team = remove_accents(game['teams']['home']['team']['name']).decode()
    away_team = remove_accents(game['teams']['away']['team']['name']).decode()
    team_df_H = df2.loc[df2['Team'] == home_team]
    team_df_A = df2.loc[df2['Team'] == away_team]
    team_df_H2 = df3.loc[df3['Team'] == home_team]
    team_df_A2 = df3.loc[df3['Team'] == away_team]

    try:
      home_team.index('Canadiens')
    except ValueError:
      try:
        home_team.index('Blues')
      except ValueError:
        team_df_H2 = df3.loc[df3['Team'] == home_team]
      else:
        team_df_H2 = df3.loc[df3['Team'] == 'St Louis Blues']
    else:
      team_df_H2 = df3.loc[df3['Team'] == 'Montreal Canadiens']
      
    try:
        away_team.index('Canadiens')
    except ValueError:
        try:
            away_team.index('Blues')
        except ValueError:
            team_df_A2 = df3.loc[df3['Team'] == away_team]
        else:
            team_df_A2 = df3.loc[df3['Team'] == 'St Louis Blues']
    else:
        team_df_A2 = df3.loc[df3['Team'] == 'Montreal Canadiens']
        
    try:
        # Calculate the values of the predictor variables for each team
        home_SH = team_df_H2['SH%'].values[0]
        home_SV = team_df_H2['SV%'].values[0]
        away_SH = team_df_A2['SH%'].values[0]
        away_SV = team_df_A2['SV%'].values[0]
    except IndexError:
        print('Data not found for one of the teams')

    try:
      home_team.index('Canadiens')
    except ValueError:
      try:
        home_team.index('Blues')
      except ValueError:
        team_df_H = df2.loc[df2['Team'] == home_team]
      else:
        team_df_H = df2.loc[df2['Team'] == 'St Louis Blues']
    else:
      team_df_H = df2.loc[df2['Team'] == 'Montreal Canadiens']
      
    try:
        away_team.index('Canadiens')
    except ValueError:
        try:
            away_team.index('Blues')
        except ValueError:
            team_df_A = df2.loc[df2['Team'] == away_team]
        else:
            team_df_A = df2.loc[df2['Team'] == 'St Louis Blues']
    else:
        team_df_A = df2.loc[df2['Team'] == 'Montreal Canadiens']
        
    try:
        # Calculate the values of the predictor variables for each team
        home_xGF = team_df_H['xGF/60'].values[0]
        home_xGA = team_df_H['xGA/60'].values[0]
        home_HDCF = team_df_H['HDCF/60'].values[0]
        home_HDCA = team_df_H['HDCA/60'].values[0]
        home_MDCF = team_df_H['MDCF/60'].values[0]
        home_MDCA = team_df_H['MDCA/60'].values[0]
        home_HDSF = team_df_H['HDSF/60'].values[0]
        home_HDSA = team_df_H['HDSA/60'].values[0]
        home_MDSF = team_df_H['MDSF/60'].values[0]
        home_MDSA = team_df_H['MDSA/60'].values[0]
        
        away_xGF = team_df_A['xGF/60'].values[0]
        away_xGA = team_df_A['xGA/60'].values[0]
        away_HDCF = team_df_A['HDCF/60'].values[0]
        away_HDCA = team_df_A['HDCA/60'].values[0]
        away_MDCF = team_df_A['MDCF/60'].values[0]
        away_MDCA = team_df_A['MDCA/60'].values[0]
        away_HDSF = team_df_A['HDSF/60'].values[0]
        away_HDSA = team_df_A['HDSA/60'].values[0]
        away_MDSF = team_df_A['MDSF/60'].values[0]
        away_MDSA = team_df_A['MDSA/60'].values[0]


    except IndexError:
        print('Data not found for one of the teams')
    # Predict the outcome of the game using the logistic regression model
    #prediction = combined_model_W( home_xGF, home_xGA, home_HDCF ,home_HDCA, home_MDCF, home_MDCA, home_PDO)
    H_GF,H_GA = combined_model_goals(home_xGF,home_xGA,home_HDCF, home_HDCA, home_MDCF, home_MDCA,home_HDSF,home_HDSA,home_MDSF,home_MDSA,home_SH,home_SV)
    A_GF,A_GA = combined_model_goals(away_xGF,away_xGA,away_HDCF, away_HDCA, away_MDCF, away_MDCA,away_HDSF,away_HDSA,away_MDSF,away_MDSA,away_SH,away_SV)
    score_H = (H_GF + A_GA)/2
    score_A = (A_GF + H_GA)/2
    total = score_H + score_A
    imp_odds_H = (score_H / total)*100
    imp_odds_A = (score_A / total)*100

    print(away_team, ':', score_A, imp_odds_A, home_team, ':', score_H, imp_odds_H, 'total:', total)

Accuracy_GF:  0.821567106283941
Accuracy_GA:  0.8301008533747091
MSE_GF:  0.1698555959652729
RMSE_GF:  0.41213540974450724
R-squared_GF:  0.941216647615658
MSE_GA:  0.15201586958520805
RMSE_GA:  0.38989212557476466
R-squared_GA:  0.9479566836583305
Carolina Hurricanes : [3.05500743] [50.02762128] New York Rangers : [3.05163397] [49.97237872] total: [6.1066414]
Columbus Blue Jackets : [3.25029713] [47.05586871] Washington Capitals : [3.65701798] [52.94413129] total: [6.90731511]
Florida Panthers : [3.47459924] [55.08613827] Philadelphia Flyers : [2.83297531] [44.91386173] total: [6.30757455]
Minnesota Wild : [2.92264825] [46.07476406] New Jersey Devils : [3.42062514] [53.92523594] total: [6.34327339]
Nashville Predators : [3.34296473] [50.9360065] Buffalo Sabres : [3.22010325] [49.0639935] total: [6.56306798]
Ottawa Senators : [2.754292] [42.498421] Boston Bruins : [3.72663584] [57.501579] total: [6.48092784]
Tampa Bay Lightning : [3.65532465] [54.58710962] Montreal Canadiens : [3.04099